<a href="https://colab.research.google.com/github/sree27-cloud/Phishing-mail-Analyze/blob/main/Copy_of_Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = {
    "ticket_id": [1,2,3,4,5,6,7,8,9,10],
    "message": [
        "Payment failed but amount deducted",
        "Unable to login with my password",
        "Delivery is delayed for my order",
        "I want a refund for my purchase",
        "App is crashing frequently",
        "Charged twice for the same order",
        "OTP not received for login",
        "Order tracking not working",
        "Refund not processed yet",
        "General query about services"
    ],
    "created_time": pd.date_range("2025-01-01", periods=10, freq="H")
}

df = pd.DataFrame(data)
df

/tmp/ipython-input-3769700383.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "created_time": pd.date_range("2025-01-01", periods=10, freq="H")


,ticket_id,message,created_time
0,1,Payment failed but amount deducted,2025-01-01 00:00:00
1,2,Unable to login with my password,2025-01-01 01:00:00
2,3,Delivery is delayed for my order,2025-01-01 02:00:00
3,4,I want a refund for my purchase,2025-01-01 03:00:00
4,5,App is crashing frequently,2025-01-01 04:00:00
5,6,Charged twice for the same order,2025-01-01 05:00:00
6,7,OTP not received for login,2025-01-01 06:00:00
7,8,Order tracking not working,2025-01-01 07:00:00
8,9,Refund not processed yet,2025-01-01 08:00:00
9,10,General query about services,2025-01-01 09:00:00


In [2]:
df.to_csv("customer_support_tickets.csv", index=False)

In [3]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df['message'].apply(clean_text)
df[['message', 'clean_message']]

,message,clean_message
0,Payment failed but amount deducted,payment failed but amount deducted
1,Unable to login with my password,unable to login with my password
2,Delivery is delayed for my order,delivery is delayed for my order
3,I want a refund for my purchase,i want a refund for my purchase
4,App is crashing frequently,app is crashing frequently
5,Charged twice for the same order,charged twice for the same order
6,OTP not received for login,otp not received for login
7,Order tracking not working,order tracking not working
8,Refund not processed yet,refund not processed yet
9,General query about services,general query about services


In [4]:
def classify_issue(text):
    if any(word in text for word in ['payment', 'charged']):
        return 'PAYMENT'
    elif any(word in text for word in ['login', 'otp', 'password']):
        return 'LOGIN'
    elif any(word in text for word in ['delivery', 'order', 'tracking']):
        return 'DELIVERY'
    elif any(word in text for word in ['refund', 'return']):
        return 'REFUND'
    elif any(word in text for word in ['crash', 'bug', 'error']):
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['clean_message'].apply(classify_issue)
df[['clean_message', 'issue_type']]

,clean_message,issue_type
0,payment failed but amount deducted,PAYMENT
1,unable to login with my password,LOGIN
2,delivery is delayed for my order,DELIVERY
3,i want a refund for my purchase,REFUND
4,app is crashing frequently,BUG
5,charged twice for the same order,PAYMENT
6,otp not received for login,LOGIN
7,order tracking not working,DELIVERY
8,refund not processed yet,REFUND
9,general query about services,GENERAL


In [5]:
def assign_priority(text):
    if any(word in text for word in ['failed', 'not working', 'crash']):
        return 'P1'
    elif any(word in text for word in ['delayed', 'slow']):
        return 'P2'
    else:
        return 'P3'

df['priority'] = df['clean_message'].apply(assign_priority)
df[['clean_message', 'priority']]

,clean_message,priority
0,payment failed but amount deducted,P1
1,unable to login with my password,P3
2,delivery is delayed for my order,P2
3,i want a refund for my purchase,P3
4,app is crashing frequently,P1
5,charged twice for the same order,P3
6,otp not received for login,P3
7,order tracking not working,P1
8,refund not processed yet,P3
9,general query about services,P3


In [6]:
from datetime import timedelta

sla_map = {'P1': 4, 'P2': 24, 'P3': 72}

df['sla_hours'] = df['priority'].map(sla_map)
df['due_time'] = df['created_time'] + pd.to_timedelta(df['sla_hours'], unit='h')

df[['priority', 'sla_hours', 'due_time']]

,priority,sla_hours,due_time
0,P1,4,2025-01-01 04:00:00
1,P3,72,2025-01-04 01:00:00
2,P2,24,2025-01-02 02:00:00
3,P3,72,2025-01-04 03:00:00
4,P1,4,2025-01-01 08:00:00
5,P3,72,2025-01-04 05:00:00
6,P3,72,2025-01-04 06:00:00
7,P1,4,2025-01-01 11:00:00
8,P3,72,2025-01-04 08:00:00
9,P3,72,2025-01-04 09:00:00


In [7]:
manager_report = df.groupby(['issue_type', 'priority']).size().reset_index(name='ticket_count')
manager_report

,issue_type,priority,ticket_count
0,BUG,P1,1
1,DELIVERY,P1,1
2,DELIVERY,P2,1
3,GENERAL,P3,1
4,LOGIN,P3,2
5,PAYMENT,P1,1
6,PAYMENT,P3,1
7,REFUND,P3,2


In [8]:
df.to_csv("final_ticket_triage_output.csv", index=False)

In [9]:
df = pd.DataFrame(data)

In [10]:
df.to_csv("final_ticket_triage_output.csv", index=False)

In [11]:
!ls

customer_support_tickets.csv  final_ticket_triage_output.csv  sample_data


In [12]:
from google.colab import files
files.download("final_ticket_triage_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>